## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-13-05-20 +0000,nyt,Bangladesh Court Sentences Former Leader Sheik...,https://www.nytimes.com/2025/11/17/world/asia/...
1,2025-11-17-13-02-00 +0000,wsj,Gibraltar to Acquire OmniMax International for...,https://www.wsj.com/business/deals/gibraltar-t...
2,2025-11-17-13-00-50 +0000,nypost,Former Fed Governor Adriana Kugler’s exit came...,https://nypost.com/2025/11/17/business/former-...
3,2025-11-17-12-59-35 +0000,bbc,Bangladesh's ousted leader Sheikh Hasina sente...,https://www.bbc.com/news/articles/cpwvg99e8vdo...
4,2025-11-17-12-58-26 +0000,nyt,Arctic Blast Brings Risk of Snow and Ice to Br...,https://www.nytimes.com/2025/11/16/weather/uk-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,35
96,epstein,14
336,new,12
97,files,10
91,up,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
104,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,102
117,2025-11-17-02-44-24 +0000,nypost,Trump says House Republicans should vote to re...,https://nypost.com/2025/11/16/us-news/trump-sa...,85
218,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,85
56,2025-11-17-10-21-07 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,80
19,2025-11-17-11-56-04 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
104,102,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
177,37,2025-11-16-21-02-08 +0000,latimes,Body of 7-year-old girl is found off Big Sur; ...,https://www.latimes.com/california/story/2025-...
145,37,2025-11-17-00-19-40 +0000,nypost,"Liberal pols, DSA warn Gov. Kathy Hochul to ba...",https://nypost.com/2025/11/16/us-news/liberals...
70,36,2025-11-17-09-57-22 +0000,cbc,Bangladesh's ousted PM Sheikh Hasina sentenced...,https://www.cbc.ca/news/world/bangladesh-tribu...
229,36,2025-11-16-13-28-00 +0000,wsj,New Rare-Earths Plant in Europe Shows How Toug...,https://www.wsj.com/world/europe/a-new-rare-ea...
158,35,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
119,29,2025-11-17-02-30-26 +0000,nypost,Trump says Venezuela would ‘like to talk’ — as...,https://nypost.com/2025/11/16/world-news/trump...
217,29,2025-11-16-17-00-00 +0000,wsj,Over 60% of Americans are comfortable with how...,https://www.wsj.com/politics/policy/economic-o...
82,28,2025-11-17-09-00-00 +0000,cbc,"Budget, government’s fate up in the air as MPs...",https://www.cbc.ca/news/politics/budget-canada...
205,27,2025-11-16-18-36-27 +0000,nyt,UN Climate Summit in Brazil Brings Tourism and...,https://www.nytimes.com/2025/11/16/world/ameri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
